In [1]:
import numpy as np
from tqdm import tqdm
from ecit import *

/opt/anaconda3/envs/graph/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
np.random.seed(1)
random.seed(1)

t = 140
n_list = [800]
ensCIT = [(knnscit, 400, p_alpha1)]#, (knnscit, 400, p_alpha175)]
zDiss = ['gaussian'] 
noiseDiss = ['t3','t1'] 

results_table = []

for n in n_list:
    for zDis in zDiss:
        for cit, k, p_ensemble in ensCIT:
            row =  [n, zDis.capitalize(), cit.__name__ + str(k) + p_ensemble.__name__]

            for noiseDis in tqdm(noiseDiss, desc=f"{n:>5}, {zDis:>8}, {k:>3},{p_ensemble.__name__:>11}"):
                eI = 0
                eII = 0
                for i in range(t):
                    retries = 0
                    while retries < 5:
                        try:
                            dataI = np.hstack((generate_samples(n=n,indp='C',z_dis=zDis, noise_dis=noiseDis,noise_std=1)))
                            dataII = np.hstack((generate_samples(n=n,indp='N',z_dis=zDis, noise_dis=noiseDis,noise_std=1)))
                            obj_ECIT = ECIT(dataI, cit, p_ensemble, k if k<100 else int(n/k))
                            pI = obj_ECIT([0], [1], [2])
                            obj_ECIT = ECIT(dataII, cit, p_ensemble, k if k<100 else int(n/k))
                            pII = obj_ECIT([0], [1], [2])
                            break
                        except Exception as e:
                            retries += 1
                            print(f"Retries times {retries}")
                            if retries >= 5: raise e
                    if pI<0.05:
                        eI += 1
                    if pII>0.05:
                        eII += 1
                eI = eI/t
                power = (t - eII)/t
                row.append(eI)
                row.append(power)
            results_table.append(row)

results_table

  800, gaussian, 400,   p_alpha1: 100%|██████████| 2/2 [16:11:02<00:00, 29131.09s/it] 


[[800,
  'Gaussian',
  'knnscit400p_alpha1',
  0.03571428571428571,
  0.39285714285714285,
  0.02857142857142857,
  0.4785714285714286]]